In [6]:
# Cell 0 — Installs (safe to re-run)
!pip install -q pandas bokeh geopy


In [7]:
# Cell 1 — Notebook setup
from bokeh.io import output_notebook
output_notebook()
print("Bokeh is ready to render inline ✅")


Loading BokehJS ...

Bokeh is ready to render inline ✅


In [10]:
# Cell 2 — Part A: Polygons with Holes
from bokeh.plotting import figure, show
from bokeh.models import Title

# Canvas
p = figure(width=700, height=700, x_range=(0, 6), y_range=(0, 5),
           title="Week 3 – Part A: Polygons with Holes")
p.add_layout(Title(text="(Holes simulated via white overlays)"), "above")

def draw_polygon_with_holes(fig, outer_xy, holes_xy, fill_color, line_color="black"):
    ox, oy = zip(*outer_xy)
    fig.patch(ox, oy, fill_color=fill_color, line_color=line_color)
    for h in holes_xy:
        hx, hy = zip(*h)
        fig.patch(hx, hy, fill_color="white", line_color="white")

# Polygon 1 (orange)
poly1_outer = [(1,3), (1,4), (2.5,4), (2.5,3)]
poly1_hole1 = [(1.2,3.2), (1.2,3.6), (1.6,3.6), (1.6,3.2)]
poly1_hole2 = [(1.6,3.8), (1.8,3.8), (1.8,3.4)]
draw_polygon_with_holes(p, poly1_outer, [poly1_hole1, poly1_hole2], fill_color="orange")

# Polygon 2 (green)
poly2_outer = [(1,1), (1,2), (2,2), (2,1)]
poly2_hole1 = [(1.3,1.3), (1.3,1.6), (1.7,1.6)]
draw_polygon_with_holes(p, poly2_outer, [poly2_hole1], fill_color="green")

# Polygon 3 (orange)
poly3_outer = [(3,1), (3,3), (5,3), (5,1)]
poly3_hole1 = [(3.5,1.5), (3.5,2.5), (4.6,2.5), (4.2,1.5)]
draw_polygon_with_holes(p, poly3_outer, [poly3_hole1], fill_color="orange")

# Style
p.grid.visible = True
p.xaxis.axis_label = "X"
p.yaxis.axis_label = "Y"
show(p)


In [12]:
# Cell 3 — Load CSV & quick diagnostics
import pandas as pd

CSV_PATH = "NSF_master_table-A2.csv"  
df = pd.read_csv(CSV_PATH)
print(f"Loaded {len(df):,} rows")
print("Columns:", list(df.columns))
df.head(3)


Loaded 12 rows
Columns: ['title', 'nsf_org', 'date_last_amended', 'award_number', 'award_instrument', 'date_started', 'date_expires', 'expected_total_amount', 'institution_name', 'inst_id', 'state', 'address', 'city', 'zipcode', 'inst_ctry_code', 'nsf_program', 'pgm_ref_codes', 'pgm_ref_names1', 'pgm_ref_names2', 'abstract', 'foa_codes', 'field_application', 'name_first_1', 'name_middle_1', 'name_last_1', 'name_first_2', 'name_middle_2', 'name_last_2', 'name_first_3', 'name_middle_3', 'name_last_3', 'name_first_4', 'name_middle_4', 'name_last_4', 'name_first_5', 'name_middle_5', 'name_last_5', 'name_first_6', 'name_middle_6', 'name_last_6', 'name_first_7', 'name_middle_7', 'name_last_7', 'email_address', 'published_year', 'id', 'last_updated']


,title,nsf_org,date_last_amended,award_number,award_instrument,date_started,date_expires,expected_total_amount,institution_name,inst_id,...,name_first_6,name_middle_6,name_last_6,name_first_7,name_middle_7,name_last_7,email_address,published_year,id,last_updated
0,US-Austria Cooperative Research: Fast Solvers...,OISE,7/3/2008,0405349,Standard Grant,9/1/2004,8/31/2009,57174,University of Kentucky Research Foundation ...,0019893001,...,NaN,NaN,NaN,NaN,NaN,NaN,craig.c.douglas@gmail.com ...,2004,92037,NaN
1,An Evaluation of the Utilization of Robotics T...,CBET,5/31/1994,9402319,Standard Grant,6/15/1994,5/31/1996,12000,University of Illinois at Chicago ...,0080010000,...,NaN,NaN,NaN,NaN,NaN,NaN,U60044@UICVM ...,1994,43255,NaN
2,Construction of Chemistry Research Facilities ...,NaN,NaN,640U606,NaN,1/1/1964,1/1/1967,514200,DATA NOT AVAILABLE ...,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1964,55123,NaN


In [14]:
# Cell 4 — Detect columns and prep for coordinates
import re

# Try existing lat/lon first
lat_col = next((c for c in df.columns if re.search(r'\blat(itude)?\b', c, re.I)), None)
lon_col = next((c for c in df.columns if re.search(r'\b(lon(gitude)?|lng)\b', c, re.I)), None)

# Choose a likely award/amount column for sizing
amt_col = next((c for c in df.columns if re.search(r'(award|amount|total)', c, re.I)), None)
if amt_col is None:
    num_cols = df.select_dtypes(include="number").columns.tolist()
    amt_col = num_cols[0] if num_cols else None

print("Detected -> lat:", lat_col, "| lon:", lon_col, "| amount:", amt_col)

# Heuristic address-like fields (for geocoding if needed)
addr_cols = [c for c in df.columns if re.search(r'address', c, re.I)]
city_cols = [c for c in df.columns if re.search(r'city', c, re.I)]
state_cols = [c for c in df.columns if re.search(r'state|province|region', c, re.I)]
zip_cols = [c for c in df.columns if re.search(r'zip|postal', c, re.I)]
country_cols = [c for c in df.columns if re.search(r'country', c, re.I)]

print("Location candidates:",
      {"address": addr_cols, "city": city_cols, "state": state_cols, "zip": zip_cols, "country": country_cols})

# Peek at the location-related columns we’ll use
preview_cols = (addr_cols[:1] + city_cols[:1] + state_cols[:1] + zip_cols[:1] + country_cols[:1])
if amt_col and amt_col not in preview_cols:
    preview_cols.append(amt_col)
df[preview_cols].head(10)


Detected -> lat: None | lon: None | amount: award_number
Location candidates: {'address': ['address', 'email_address'], 'city': ['city'], 'state': ['state'], 'zip': ['zipcode'], 'country': []}


,address,city,state,zipcode,award_number
0,201 Kinkead Hall,Lexington,KY,40506,0405349
1,809 S MARSHFIELD RM 608,CHICAGO,IL,60612,9402319
2,,,,,640U606
3,4800 Calhoun Boulevard,Houston,TX,77204,7812060
4,,,,,6216560
5,,,,,65U1059
6,Room 424 Rangos Building,Pittsburgh,PA,15282,0511444
7,106 New Scotland Avenue,Albany,NY,12208,9850530
8,106 New Scotland Avenue,Albany,NY,12208,8750088
9,,,,,65U1134


In [16]:
# Cell 5 — Build lat/lon
import os, time
from geopy.geocoders import Nominatim

GEOCACHE_PATH = "geocache.csv"
RATE_SECONDS = 1.2  # rate limit to be polite

def build_full_address(row):
    parts = []
    for cols in (addr_cols, city_cols, state_cols, zip_cols, country_cols):
        if cols:
            val = str(row[cols[0]]).strip()
            if val and val.lower() != "nan":
                parts.append(val)
    return ", ".join(parts) if parts else None

if lat_col and lon_col:
    # Use existing coordinates
    df["lat"] = pd.to_numeric(df[lat_col], errors="coerce")
    df["lon"] = pd.to_numeric(df[lon_col], errors="coerce")
else:
    # Build addresses to geocode
    df["__full_address"] = df.apply(build_full_address, axis=1)
    df = df[df["__full_address"].notna()].copy()

    # Load or init cache
    if os.path.exists(GEOCACHE_PATH):
        cache = pd.read_csv(GEOCACHE_PATH).drop_duplicates("address", keep="last")
    else:
        cache = pd.DataFrame(columns=["address","lat","lon"])
    cached = dict(zip(cache["address"], zip(cache["lat"], cache["lon"])))

    geolocator = Nominatim(user_agent="nsf_mapping_assignment")

    def safe_geocode(addr):
        if addr in cached:
            return cached[addr]
        try:
            loc = geolocator.geocode(addr, timeout=20)
            time.sleep(RATE_SECONDS)
            if loc:
                cached[addr] = (loc.latitude, loc.longitude)
                return cached[addr]
        except Exception:
            pass
        return (None, None)

    df["lat"], df["lon"] = zip(*df["__full_address"].map(safe_geocode))

    # Persist cache
    cache_df = pd.DataFrame([{"address": a, "lat": v[0], "lon": v[1]} for a, v in cached.items()])
    cache_df.to_csv(GEOCACHE_PATH, index=False)

# Keep only rows with valid coordinates
df = df.dropna(subset=["lat","lon"]).copy()

# Prepare amount for sizing
if amt_col:
    df["amount"] = pd.to_numeric(df[amt_col], errors="coerce").fillna(0)
else:
    df["amount"] = 0

print("Rows with coordinates:", len(df))
df[["lat","lon","amount"]].head(10)


Rows with coordinates: 3


,lat,lon,amount
7,42.652026,-73.778614,9850530
8,42.652026,-73.778614,8750088
11,39.948264,-75.209132,9616948


In [17]:
# Cell 6 — Plot proportional symbol map
import math
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.tile_providers import get_provider, CARTODBPOSITRON

def lonlat_to_webmercator(lon, lat):
    k = 6378137.0
    x = math.radians(lon) * k
    y = math.log(math.tan(math.pi/4 + math.radians(lat)/2)) * k
    return x, y

merc = df.apply(lambda r: lonlat_to_webmercator(r["lon"], r["lat"]), axis=1, result_type="expand")
df["x"], df["y"] = merc[0], merc[1]

def size_scale(series, min_px=6, max_px=40):
    s = series.clip(lower=0)
    if s.max() == s.min():
        return pd.Series(min_px, index=s.index)
    norm = (s - s.min()) / (s.max() - s.min())
    return min_px + norm * (max_px - min_px)

df["size"] = size_scale(df["amount"])

# Tooltips: add friendly fields if present
tooltips = [("Award Amount", "@amount{($ 0.00 a)}"),
            ("Lat, Lon", "@lat{0.000}, @lon{0.000}")]
for name in ["Organization", "Institution", "Org Name", "Principal Investigator",
             "PI Name", "Project Title", "Title"]:
    if name in df.columns:
        tooltips.insert(0, (name, f"@{{{name}}}"))

source = ColumnDataSource(df)

p = figure(
    title="Week 3 – Part B: NSF Proportional Symbol Map",
    x_axis_type="mercator", y_axis_type="mercator",
    width=900, height=600
)
p.add_tile(get_provider(CARTODBPOSITRON))
p.circle(x="x", y="y", size="size", source=source, fill_alpha=0.6, line_color="white")
p.add_tools(HoverTool(tooltips=tooltips))
p.xaxis.visible = False
p.yaxis.visible = False
show(p)
